# 1. Preparation

### 1.1 Packages, Dates, & Initials

In [1]:
#administrative packages
import os
import datetime
mydate = datetime.datetime.now()
from datetime import date,timedelta
import time
start = time.time()

#standard packages
import pandas as pd
import numpy as np

#administrative packages
import os
import datetime
import random
mydate = datetime.datetime.now()

proj_path = 'O:\CDD\PLANNING\GIS\Projects\Permit_Metrics'
os.chdir(proj_path)

%load_ext autoreload
%autoreload 2

from functions.admin import * 
from functions.permits import *
from functions.process import *

%load_ext autoreload
%autoreload 2

np.warnings.filterwarnings(
    'ignore', 
    category=np.VisibleDeprecationWarning) 
pd.options.mode.chained_assignment = None

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
holidays = get_holidays()

file_name = 'PermitMetrics_Planning.csv'
tit = 'Planning'
dept = tit

### 1.2 Finding Type

In [3]:
local_folder = proj_path
#local_folder = r'C:\Users\nelms\Dropbox\Python\WC'

os.chdir(local_folder)
file_path = os.path.join(os.getcwd(), 'data', 'raw_data', file_name)



less, great = ('2017Q4', '2023Q1')
less, great = ((pd.to_datetime(less)).to_period('Q'), (pd.to_datetime(great)).to_period('Q'))

# 2. Preparing Admin & Count Dataframes

In [4]:
raw_permit_df = pd.read_csv(file_path)
raw_permit_df = first_clean(raw_permit_df)
raw_permit_df = fix_ents(pd.read_csv(file_path))
raw_permit_df = group_entitlments(raw_permit_df)
#raw_permit_df = get_decisions(raw_permit_df)
raw_permit_df = get_board_info(raw_permit_df)
raw_permit_df = get_planner(raw_permit_df)
raw_permit_df = clean_addresses(raw_permit_df)

sa_filt = (~((raw_permit_df['Task']=='Staff Analysis')&(raw_permit_df['Status']=='Deemed Complete')))
raw_permit_df = raw_permit_df[sa_filt]

prep_permit_df = raw_permit_df[raw_permit_df['Primary']=='Yes']

date_cols = [c for c in prep_permit_df.columns if 'Date' in c]
prep_permit_df[date_cols] = prep_permit_df[date_cols].apply(pd.to_datetime, errors='coerce')

prep_permit_df = add_task2rank(prep_permit_df)
prep_permit_df.sort_values(sorts, inplace=True)
prep_permit_df.reset_index(drop=True, inplace=True)
prep_permit_df = add_phase_and_action(prep_permit_df)


prep_permit_df = get_task_ids(prep_permit_df)
pdict = get_permit_dictionary(prep_permit_df)
pids = list(pdict.keys())
prep_permit_df = fix_back2back_end_actions(prep_permit_df.copy(), pids=pids)

prep_permit_df = get_rounds(prep_permit_df)
prep_permit_df = get_decision_type(prep_permit_df)

Formatted_Permits, Admin_Permits = prepare_times(prep_permit_df.copy(), extra_list=ent_cols)

gcol = [
    #'Record Type',
    'Permit Type',
    'Description',
    #'Notes',
    'Address',
    'Planner',
    'Record Status',
    'Open Date',
    'Date Status',
    'Primary',
    'Entitlements',
    'Num of Entitlements',
    'Boards', 
    'Board Count',
    'Staff Decision',
    'Public Decision'
 ]
Admin_Permits = \
    Admin_Permits[gcol] \
    .reset_index() \
    .rename(columns={
        'Open Date':'Date Opened',
        'Application #':'P_ID'
    })

# 3. Aggregate Tasks

### 1. Review Round Dates & Length (in Days)

In [5]:
def get_total_days_df(prep_permit_df):
    rev_round_dates = get_review_round_dates(prep_permit_df)
    
    rev_tot_dates = get_review_total_dates(rev_round_dates)
    
    dec_dates = get_decision_dates(prep_permit_df)

    # join rev_tot_dates and dec_dates
    tot_dates = \
        rev_tot_dates \
            .merge(dec_dates, on='P_ID', how='outer')

    # FIX NANS
    ## fix start dates
    nan_filt = tot_dates['Start Date'].isna()
    nan_rows = set(tot_dates.loc[nan_filt].index)
    nan_max = prep_permit_df \
        .loc[prep_permit_df['P_ID'].isin(nan_rows)] \
        .groupby('P_ID')['Date Status'] \
        .min() \
        .to_dict()

    tot_dates.loc[nan_filt, 'Start Date'] = \
        tot_dates.loc[nan_filt].index \
        .map(nan_max)

    ## fix dates
    dates_cols = [c for c in tot_dates.columns if 'Dates' in c]
    tot_dates.loc[nan_filt, dates_cols] = \
        tot_dates.loc[nan_filt, dates_cols] \
            .apply(lambda x: [[] for _ in range(len(x))], axis=0)

    ## fix rounds
    tot_dates['Rounds'] = tot_dates['Rounds'].fillna(0).astype(int)

    
    tot_dates['Review Days'] = \
        tot_dates['Review Dates'] \
        .apply(len)
    tot_dates['Staff Days'] = \
        tot_dates['Review Staff Dates'] \
        .apply(len)
    tot_dates['Customer Days'] = \
        tot_dates['Review Days'] - tot_dates['Staff Days']
    tot_dates['Decision Days'] = \
        tot_dates['Decision Dates'] \
        .apply(len)
    tot_dates['Total Days'] = \
        tot_dates['Decision Days'] + tot_dates['Review Days']
    
    #tot_dates['Decision Days'] = \
    #    tot_dates['Total Days'] - tot_dates['Review Days']
    #tot_dates.loc[
    #    tot_dates['CDecision Days'] < tot_dates['Decision Days'], 
    #    'Decision Days'] = \
    #    tot_dates['CDecision Days']
    #tot_dates.drop(columns=['CDecision Days'], inplace=True)

    #tot_dates['Total Biz Days'] = \
    #    tot_dates \
    #    .apply(
    #        lambda x: len(sum_days(x['Start Date'], x['Decision Stop Date'], 
    #                        business_days=True)), axis=1)
    tot_dates['Review Biz Days'] = \
        tot_dates['Review Biz Dates'] \
        .apply(len)
    tot_dates['Staff Biz Days'] = \
        tot_dates['Review Staff Biz Dates'] \
        .apply(len)
    tot_dates['Customer Biz Days'] = \
        tot_dates['Review Biz Days'] - tot_dates['Staff Biz Days']
    tot_dates['Decision Biz Days'] = \
        tot_dates['Decision Biz Dates'] \
        .apply(len)
    tot_dates['Total Biz Days'] = \
        tot_dates['Decision Biz Days'] + tot_dates['Review Biz Days']
    #tot_dates['Decision Biz Days'] = \
    #    tot_dates['Total Biz Days'] - tot_dates['Review Biz Days']
    #tot_dates.loc[
    #    tot_dates['CDecision Biz Days'] < tot_dates['Decision Biz Days'], 
    #    'Decision Biz Days'] = \
    #    tot_dates['CDecision Biz Days']
    #tot_dates.drop(columns=['CDecision Biz Days'], inplace=True)


    tot_dates.drop(columns=['Review Days', 'Review Biz Days'], inplace=True)
    
    tot_cols = ['Total Days', 'Total Biz Days']
    days_cols = [
        'Staff Days', 'Customer Days', 'Decision Days',
        'Staff Biz Days', 'Customer Biz Days','Decision Biz Days']
    for tc in tot_cols:
        if 'Biz' in tc:
            dcols = [c for c in days_cols if 'Biz' in c]
        else:
            dcols = [c for c in days_cols if 'Biz' not in c]
        tot_dates.loc[tot_dates[tc]==0, tc] = \
            tot_dates.loc[tot_dates[tc]==0, dcols].sum(axis=1)

        if len(tot_dates.loc[tot_dates[tc]==0]) > 0:
            tot_dates.loc[tot_dates[tc]==0, tc] = 1
    
    #tot_dates['Pct Review'] = \
    #    tot_dates['Review Days'] / tot_dates['Total Days']
    tot_dates['Pct Staff'] = \
        tot_dates['Staff Days'] / tot_dates['Total Days']
    tot_dates['Pct Customer'] = \
        tot_dates['Customer Days'] / tot_dates['Total Days']
    tot_dates['Pct Decision'] = \
        tot_dates['Decision Days'] / tot_dates['Total Days']

    tot_dates = tot_dates[[c for c in list(tot_dates) if 'Dates' not in c]]

    # flag errors
    tot_dates['Error'] = False
    tot_dates.loc[tot_dates['Total Days']<0, 'Error'] = True
    tot_dates.loc[tot_dates['Rounds']<=0, 'Error'] = True

    error_filt = tot_dates['Error']==True
    tot_dates.loc[error_filt, 'Total Biz Days'] = \
        tot_dates.loc[error_filt] \
            .apply(
                lambda x: len(sum_days(x['Start Date'], x['Decision Stop Date'], 
                            business_days=True)), axis=1)
    
    tot_dates.loc[error_filt, 'Total Days'] = \
        (
        tot_dates.loc[error_filt, 'Decision Stop Date'] - 
        tot_dates.loc[error_filt, 'Start Date']).dt.days
    
    tot_dates['Stop Date'] = tot_dates['Decision Stop Date']
    tot_dates.drop(columns=['Decision Start Date', 'Decision Stop Date'], inplace=True)

    return tot_dates

tot_dates = get_total_days_df(prep_permit_df)

full_df = \
    Admin_Permits  \
        .merge(tot_dates, on='P_ID', how='left') \
        .sort_values(['Error', 'P_ID'])

type_cols = [
    'Design Review?',
    'DR Resi?',
    'DR Comm?',
    'UP Con?',
    'UP Admin?'
]
for c in type_cols:
    full_df[c] = False

    ctype = c.replace('?','')
    if ctype in nicknames.keys():
        ctype = nicknames[ctype]

    # search the 'Entitlements' column for the type
    full_df.loc[full_df['Entitlements'].str.contains(ctype), c] = True

full_df['Date Opened'] = \
    full_df['Start Date']
full_df['Date Status'] = \
    full_df['Stop Date']
full_df.drop(columns=['Start Date', 'Stop Date'], inplace=True)

# if 'Public Decision' is false, then 'Staff Decision' is true
filt = full_df['Public Decision']=='Yes'
full_df.loc[filt, 'Staff Decision'] = 'No'

full_df \
    .to_csv('data/clean_data/planning_permit_metrics2.csv', index=False)

In [7]:
full_df['Boards'].value_counts()

               175
PC              38
DRC             37
ZA              16
DRC, PC          2
PC, DRC          2
DRC, CC, ZA      1
DRC, PC, CC      1
PC, CC           1
ZA, DRC          1
CC, PC           1
CC               1
PC, CC, DRC      1
Name: Boards, dtype: int64

In [8]:
list(full_df)

['P_ID',
 'Permit Type',
 'Description',
 'Address',
 'Planner',
 'Record Status',
 'Date Opened',
 'Date Status',
 'Primary',
 'Entitlements',
 'Num of Entitlements',
 'Boards',
 'Board Count',
 'Staff Decision',
 'Public Decision',
 'Rounds',
 'Staff Days',
 'Customer Days',
 'Decision Days',
 'Total Days',
 'Staff Biz Days',
 'Customer Biz Days',
 'Decision Biz Days',
 'Total Biz Days',
 'Pct Staff',
 'Pct Customer',
 'Pct Decision',
 'Error',
 'Design Review?',
 'DR Resi?',
 'DR Comm?',
 'UP Con?',
 'UP Admin?']

In [18]:
full_df['DRC?'] = 'Did not go to DRC'
full_df.loc[full_df['Boards'].str.contains('DRC'), 'DRC?'] = 'Went to DRC'

drc_df = \
    full_df \
        .groupby('DRC?') \
        .agg({
            'P_ID':'count',
            'Total Days':'mean', 'Rounds':'mean', 'Decision Days':'mean'})

# make a new row that is the difference of the other two rows
drc_df.loc['Difference'] = \
    drc_df.loc['Went to DRC'] - drc_df.loc['Did not go to DRC']

drc_df \
    .round(1)

,P_ID,Total Days,Rounds,Decision Days
DRC?,,,,
Did not go to DRC,232.0,118.7,1.5,28.7
Went to DRC,45.0,211.0,2.0,42.2
Difference,-187.0,92.3,0.5,13.5


In [25]:
full_df['DRC?'] = 'Did not go to DRC'
full_df.loc[full_df['Boards'].str.contains('DRC'), 'DRC?'] = 'Went to DRC'

drc_df = \
    full_df \
        .loc[full_df['Entitlements'].str.contains('DR')] \
        .groupby('Staff Decision') \
        .agg({
            'P_ID':'count',
            'Total Days':'mean', 'Rounds':'mean', 'Decision Days':'mean'})


# make a new row that is the difference of the other two rows
drc_df.loc['Difference'] = \
    drc_df.loc['No'] - drc_df.loc['Yes']

drc_df \
    .round(1)

,P_ID,Total Days,Rounds,Decision Days
Staff Decision,,,,
No,43.0,202.5,2.3,55.7
Yes,109.0,166.3,1.6,31.3
Difference,-66.0,36.3,0.6,24.4


In [26]:
drc_df = \
    full_df \
        .loc[full_df['Permit Type'].str.contains('Design Review')] \
        .groupby('Permit Type') \
        .agg({
            'P_ID':'count',
            'Total Days':'mean', 'Rounds':'mean', 'Decision Days':'mean'})


# make a new row that is the difference of the other two rows
#drc_df.loc['Difference'] = \
#    drc_df.loc['No'] - drc_df.loc['Yes']

drc_df \
    .round(1)

,P_ID,Total Days,Rounds,Decision Days
Permit Type,,,,
Design Review Commercial,80,161.7,1.6,45.3
Design Review Other,6,196.7,1.7,125.7
Design Review Residential,38,171.4,2.1,9.2
Design Review Sign,18,198.6,1.8,30.2


In [35]:
rev_round_dates = get_review_round_dates(prep_permit_df)
focus_id = "Y18-026"
tdf = rev_round_dates \
    .loc[rev_round_dates.index.get_level_values("P_ID")==focus_id]
tdf

,,Start Date,Stop Date,Dates,Biz Dates,Days,Biz Days
P_ID,Round,,,,,,
Y18-026,1,2018-03-15,2018-05-18,"[2018-03-15 00:00:00, 2018-03-16 00:00:00, 201...","[2018-03-15 00:00:00, 2018-03-16 00:00:00, 201...",65,47


In [35]:
start = tdf['Start Date'].to_list()[1:]
stop = tdf['Stop Date'].to_list()[:-1]
ss = [
    len(sum_days(t,p, business_days=False))-2
    for t,p in list(zip(stop, start))
]
print(ss)
print(sum(ss))

[139, 18, 2]
159


In [19]:
len(sum_days("2019-08-08", "2019-10-16", business_days=False))

70

### 3. Decision Dates

### 4. Boards

In [ ]:
board_dec_t = [
    t for t in public_dec_t if t != 'Zoning Administrator'
]

def get_boards(
        group, 
        board_list = board_dec_t,
        filt_date_col='Date Status',
        filt_col='Phase', 
        filt_val='Decision'
        ):

    yes_filt = group[filt_col] == filt_val
    round_filt = group['Round'] > 0
    max_no_date = group.loc[(~yes_filt)&(round_filt), filt_date_col].max()
    filt_group = group[yes_filt&(group[filt_date_col] >= max_no_date)].copy()

    return [t for t in filt_group['Task'].to_list() if t in board_list]

board_df = \
    prep_permit_df \
        .groupby('P_ID') \
        .apply(get_boards) \
        .to_frame() \
        .rename(columns= {0: 'Boards List'})
board_df['Boards'] = board_df['Boards List'].apply(len)

board_df[board_df['Boards']>0].head(10)

In [ ]:
def count_actions(a_list, actions=['start','end', 'round start','round end']):
    a_dict = {a:0 for a in actions}

    for find_action in actions:
        for raw_action in a_list:
            if find_action == raw_action:
                a_dict[find_action] += 1

    return pd.Series([a_dict[a] for a in actions], index=actions)
# check if overlap
test_df = \
    prep_permit_df \
        .copy() \
        [prep_permit_df['Phase']=='Review'] \
        .groupby('P_ID') \
        ['Action'].apply(count_actions) \
        .unstack()

test_df['start_end_dif'] = test_df['start'] - test_df['end']
too_much = test_df.loc[test_df['start_end_dif']!=0].index
too_much = [t for t in too_much if t not in bad]

remove = 'Staff Analysis|Deemed Complete'
'Y22-111' #consolidated comments is last

if len(too_much)>0:
    num = too_much[0]
    focus_df = test_df[test_df.index==num]
    print('start/end dif: ', focus_df.loc[focus_df['start_end_dif']!=0].shape[0])
    preview_row(num, prep_permit_df)

In [ ]:

test_df['round_start_end_dif'] = test_df['round start'] - test_df['round end']
too_much = test_df.loc[test_df['round_start_end_dif']!=0].index
too_much = [t for t in too_much if t not in bad]

remove = 'Staff Analysis|Deemed Complete'
'Y22-111' #consolidated comments is last

if len(too_much)>0:
    num = too_much[0]
    focus_df = test_df[test_df.index==num]
    print('start/end dif: ', focus_df.loc[focus_df['round_start_end_dif']!=0].shape[0])
    preview_row(num, prep_permit_df)

,,Start Date,Stop Date,Days,Biz Days
P_ID,Round,,,,
Y22-080,1,2022-08-30,2022-09-28,30,21
Y22-083,1,2022-09-07,2022-10-04,28,20
Y22-084,1,2022-09-07,2022-10-04,28,20
Y22-085,1,2022-09-12,2022-10-04,23,17
Y22-089,1,2022-09-21,2022-10-19,29,21
Y22-092,1,2022-09-28,2022-10-26,29,21
Y22-095,1,2022-10-03,2022-10-19,17,13
Y22-097,1,2022-10-06,2023-01-31,118,73
Y22-098,1,2022-10-12,2022-11-16,36,25


In [22]:

ts = prep_permit_df['Task Status'].value_counts()

ts[ts.index.str.contains('Staff Level Decision')].index
prep_permit_df \
    [prep_permit_df['Phase'] == 'Decision'] \
    ['Task'].value_counts() \
    .index

# filter prep_permit_df with P_IDs that have one of their tasks in the public_dec_t list

public_dec_pids = \
    prep_permit_df \
        [prep_permit_df['Task'].isin(public_dec_t)] \
        ['P_ID'].unique()

#prep_permit_df[prep_permit_df['P_ID'].isin(public_dec_pids[0:2])]

KeyError: 'Task Status'

In [ ]:
focus_id = pids[10]
fpids = public_dec_pids #pids
gpid = [i for i in fpids if i not in bad+sorta_bad]
num = random.randint(0, len(gpid))
focus_id = gpid[num]
focus_id = 'Y19-002'

bad_filt = (prep_permit_df['P_ID'].isin(bad+sorta_bad))
phase_filt = (prep_permit_df['Phase']=='Review')
focus_filt = (prep_permit_df['P_ID']==focus_id)
search_df = prep_permit_df[focus_filt].copy()

print(*[ii for ii in find_round(search_df)], sep = ', ')
print('Staff Decision: {} | Public Decision: {}'.format(
    search_df['Staff Decision'].unique()[0], 
    search_df['Public Decision'].unique()[0]
))
preview_row(focus_id, prep_permit_df, check_df=Records.copy(), pids=gpid)


1, 1, 1, 1, 0, 0, 0
Staff Decision: No | Public Decision: Yes
Y19-002 | UP Conditional
Total 42 | Rounds 1 | Days per Round 18


T_ID Date Assigned  \
Phase    Task                  Status                                    
Review   Intake Review         Application Accepted  001    2019-01-09   
         Planning Review       Comments              002    2019-01-24   
         Consolidated Comments Deemed Complete       003    2019-01-24   
         Staff Analysis        Set for Hearing       004    2019-03-19   
Decision Planning Commission   Approved              005    2019-03-19   
Review   CEQA Review           Project Exempt        006           NaT   
Decision Close Out             Approved - Closed     007    2019-03-19   

                                                    Date Status Action  
Phase    Task                  Status                                   
Review   Intake Review         Application Accepted  2019-01-24  start  
         Planning Review       Comments              2019-01-24         
         Consolidated Comments Deemed Complete       2019-02-19         
         Staff Analysis        Set for Hearing       2019-02-19    end  
Decision Planning Commission   Approved              2019-03-14         
Review   CEQA Review           Project Exempt        2019-03-19         
Decision Close Out             Approved - Closed     2019-03-19

In [ ]:
focus_id = pids[10]
fpids = pids
gpid = [i for i in fpids if i not in bad+sorta_bad]
num = random.randint(0, len(gpid))
focus_id = gpid[num]
focus_id = 'Y19-002'

bad_filt = (prep_permit_df['P_ID'].isin(bad+sorta_bad))
phase_filt = (prep_permit_df['Phase']=='Review')
focus_filt = (prep_permit_df['P_ID']==focus_id)
search_df = prep_permit_df[focus_filt].copy()

print(*[ii for ii in find_round(search_df)], sep = ', ')
print('Staff Decision: {} | Public Decision: {}'.format(
    search_df['Staff Decision'].unique()[0], 
    search_df['Public Decision'].unique()[0]
))
preview_row(focus_id, prep_permit_df, check_df=Records.copy(), pids=gpid)


1, 1, 1, 1, 0, 0, 0
Staff Decision: No | Public Decision: Yes
Y19-002 | UP Conditional
Total 42 | Rounds 1 | Days per Round 18


T_ID Date Assigned  \
Phase    Task                  Status                                    
Review   Intake Review         Application Accepted  001    2019-01-09   
         Planning Review       Comments              002    2019-01-24   
         Consolidated Comments Deemed Complete       003    2019-01-24   
         Staff Analysis        Set for Hearing       004    2019-03-19   
Decision Planning Commission   Approved              005    2019-03-19   
Review   CEQA Review           Project Exempt        006           NaT   
Decision Close Out             Approved - Closed     007    2019-03-19   

                                                    Date Status Action  
Phase    Task                  Status                                   
Review   Intake Review         Application Accepted  2019-01-24  start  
         Planning Review       Comments              2019-01-24         
         Consolidated Comments Deemed Complete       2019-02-19         
         Staff Analysis        Set for Hearing       2019-02-19    end  
Decision Planning Commission   Approved              2019-03-14         
Review   CEQA Review           Project Exempt        2019-03-19         
Decision Close Out             Approved - Closed     2019-03-19

# 3. Main Day & Round Processing

In [10]:
Records, rounds_dict = get_counts(Formatted_Permits)

Records = Records.reset_index()#[keep_fields]

these ids had an index error []


## 4.1 Board Counts

In [40]:
join_id = 'Application #'
comm_permit_df = (
    get_comm_days(
        prep_permit_df.copy(), 
        final_day_dict = Records.set_index(join_id)['Final Date'].apply(pd.to_datetime).to_dict()
    )
    .set_index('P_ID')
    )
comm_permit_df.index.name = join_id

{'Y18-105': {'id': 'Y18-105', 'start_date': Timestamp('2019-03-04 00:00:00'), 'end_date': Timestamp('2019-03-04 00:00:00'), 'final_date': Timestamp('2019-03-04 00:00:00')}, 'Y18-113': {'id': 'Y18-113', 'start_date': Timestamp('2019-06-25 00:00:00'), 'end_date': Timestamp('2019-06-25 00:00:00'), 'final_date': Timestamp('2019-05-09 00:00:00')}, 'Y19-024': {'id': 'Y19-024', 'start_date': Timestamp('2019-09-16 00:00:00'), 'end_date': Timestamp('2019-09-18 00:00:00'), 'final_date': Timestamp('2019-09-16 00:00:00')}, 'Y20-043': {'id': 'Y20-043', 'start_date': Timestamp('2020-08-12 00:00:00'), 'end_date': Timestamp('2020-08-12 00:00:00'), 'final_date': Timestamp('2020-07-09 00:00:00')}, 'Y19-121': {'id': 'Y19-121', 'start_date': Timestamp('2019-12-18 00:00:00'), 'end_date': Timestamp('2020-03-04 00:00:00'), 'final_date': Timestamp('2020-03-04 00:00:00')}, 'Y22-022': {'id': 'Y22-022', 'start_date': Timestamp('2022-04-25 00:00:00'), 'end_date': Timestamp('2022-05-05 00:00:00'), 'final_date': Ti

In [41]:
board_cols = ['Board Total Days', 'Board Business Days', 'Board Start Date', 'Board End Date']

# Join the last four columns of comm_permit_df to Admin_Permits on P_ID but make sure those four columns aren't in the columns of Admin_Permits before joining
Admin_Permits = (
    Admin_Permits 
    [[l for l in list(Admin_Permits) if l not in board_cols]]
    .reset_index()
    .merge(
        comm_permit_df[['Board Total Days','Board Business Days','Board Start Date','Board End Date']], 
        on=join_id, how='left')
    .set_index(join_id)
    )

In [42]:
def get_reviews(file_path, let=False):
    let = let or False
    bb = prepare_times(file_path)
    bbb = bb.loc[(bb['Task'].str.contains('Review'))&(bb['Status']=='Routed')].reset_index()[['#','Task Status', 'Task']].groupby(['#','Task Status']).count()
    bbbi = set(bbb.index.get_level_values(0))

    if let == True:
        bb_dict = {b:[''.join(sorted([b[0] for b in bbb.xs(b).index.values]))] for b in bbbi}
    else:
        bb_dict = {b:[len(bbb.xs(b))] for b in bbbi}

    tl = 1
    while len(bbb.loc[bbb['Task']>tl, 'Task']):
        if let == True:
            [bb_dict[b].append(''.join(sorted([b[0] for b in bbb.loc[bbb['Task']>tl].xs(b).index.values]))) 
             for b in set(bbb.loc[bbb['Task']>tl].index.get_level_values(0))]
        else:
            [bb_dict[b].append(len(bbb.loc[bbb['Task']>tl].xs(b))) 
             for b in set(bbb.loc[bbb['Task']>tl].index.get_level_values(0))]
        tl = tl + 1
    return bb_dict

## 4.2 Entitlement Types

In [43]:
ent_path = r"C:\Users\nelms\OneDrive - walnut-creek.org\Projects\Permit Metrics\entitlement_counts.xlsx"
ent_df = pd.read_excel(ent_path, skiprows=1)

ent_df = ent_df[ent_df['Permit Type'].isin(Admin_Permits['Permit Type'].unique())]

ent_name_dict = {
    'Design Review Antenna':'Design Review Antenna',
    'Drip Line Encroachment':'Drip Line Encroachment',
    'MSP': 'MSP',
    'Special Use Permit (SUP)': 'SUP',
    'Tree Dripline Encroachment': 'Tree Dripline Encroachment',
    'Tree Removal Permit': 'Tree Removal Permit'
}
def add_list_to_dict(old_dict, new_list, fill=False):
    new_dict = old_dict.copy()
    for i in new_list:
        if i not in new_dict.keys():
            if fill == False:
                new_dict[i] = i
            else:
                new_dict[i] = fill
    return new_dict

ent_name_dict = add_list_to_dict(ent_name_dict, list(ent_df['Permit Type'].unique()))

ent_df['Permit Type'] = ent_df['Permit Type'].map(ent_name_dict)

def switch_dict_kv(old_dict):
    new_dict = {}
    for k,v in old_dict.items():
        if v not in new_dict.keys():
            new_dict[v] = [k]
        else:
            new_dict[v].append(k)
    return new_dict

ent_dict = \
    add_list_to_dict(
        ent_df.set_index('Permit Type')['Entitlement?'].to_dict(),
        ent_df['Permit Type'].unique(),
        fill='MAYBE'
    )
ent_dict_r = switch_dict_kv(ent_dict)
pub_dict = \
    add_list_to_dict(
        ent_df.set_index('Permit Type')['Public Hearing?'].to_dict(),
        ent_df['Permit Type'].unique(),
        fill='MAYBE'
    )
pub_dict_r = switch_dict_kv(pub_dict)

ent_df

,Permit Type,Total Counts,2018,2019,2020,2021,2022,Entitlement?,Public Hearing?,Notes
1,Design Review Commercial,81,21,28,13,17,2,YES,YES,NaN
2,Design Review Other,23,9,7,3,3,1,YES,YES,NaN
3,Design Review Residential,39,10,7,9,8,5,YES,MAYBE,NaN
5,General Plan Amendment,2,1,0,0,1,0,YES,YES,NaN
6,Hillside Performance Standards,2,1,1,0,0,0,YES,YES,NaN
13,Tentative Map Condo Conversion,1,0,1,0,0,0,YES,YES,NaN
14,Tentative Map Major Subdivision,4,2,1,1,0,0,YES,YES,NaN
15,Tentative Map Minor Subdivision,4,2,1,1,0,0,YES,YES,NaN
18,Use Permit Administrative,50,10,15,3,13,9,YES,NO,NaN
19,Use Permit Conditional,40,7,19,7,3,4,YES,YES,NaN


## 4.2 Attach Permit Information

In [44]:
def Commission_Counts(Record_Status, Record_Sums):
    temp = Record_Status.reset_index()
    hearings = set(temp.loc[temp['Task Status']=='Staff Analysis - Set for Hearing', '#'].unique())
    commissions = {
        'City Council':'CC',
           'Planning Commission':'PC',
           'Design Review Commission':'DR'
    }
    Comms = Record_Status.loc[(Record_Status.index.get_level_values(0).isin(hearings))&((Record_Status.index.get_level_values(1).str.contains('Council'))|(Record_Status.index.get_level_values(1).str.contains('Commission')))].reset_index().groupby('#').agg({'Task Status':list})
    Comms['Comm List'] = Comms['Task Status'].apply(lambda x: [commissions[c.split(' - ')[0]] for c in x])
    Comms['Commissions'] = Comms['Comm List'].apply(lambda x: len(x))
    Comms['Comm Days'] = 0
    Comms = Comms[['Commissions', 'Comm Days','Comm List']]
    Record_Sums = Record_Sums.join(Comms, how='outer')
    Record_Sums['Commissions'] = Record_Sums['Commissions'].fillna(value=0).astype(int)
    Record_Sums['Comm Days'] = Record_Sums['Comm Days'].fillna(value=0).astype(int)
    Record_Sums['Comm List'] = Record_Sums['Comm List'].fillna(value='')
    return Record_Sums
    
def Add_Commission_to_Total(Record_Status, Record_Sums):
    for index, row in Record_Sums.iterrows():
        REC = Record_Status.xs(index).sort_values(by=sorts)
        check = set(REC.iloc[[0,-1]]['Date Status'].values)
        if len(check) <= 1:
            (end,) = check
            strt = end
        else:
            strt, end = check
        COMM_strt = row['Final Date']
        if COMM_strt == end or row['Commissions']==0 :
            COMM_DAYS = 0
        else:
            COMM_DAYS = len(sum_days(COMM_strt, end, business_days=False))
            Record_Sums.loc[Record_Sums.index==index, 'Final Date'] = end
            Record_Sums.loc[Record_Sums.index==index, 'Comm Days'] = COMM_DAYS
        TOTAL_DAYS = len(sum_days(strt, end, business_days=False))
        Record_Sums.loc[Record_Sums.index==index,'Total Days'] = TOTAL_DAYS
    return Record_Sums

def quarter_round(x):
    r = round(x*4)/4
    if int(r) == r and isinstance(r, float):
        r = int(r)
    return r
def find_plural(x):
    if x > 1:
        return "s"
    else:
        return ""
def Process_Length(Days):
    if (Days < 5) and (Days != 1): 
        return str(Days) + " days"
    elif (Days == 1):
        return "1 day"
    else:
        if Days < 30:
            Weeks = int(Days/7)
            if Weeks == 0:
                Weeks = 1
            multi = find_plural(Weeks)
            return str(Weeks) + " week" + find_plural(Weeks)
        elif (Days >= 30)&(Days < 360):
            Months = int(Days/30) 
            return str(Months) + " month" + find_plural(Months)
        else:
            Years = quarter_round(Days/365)
            return str(Years) + " year" + find_plural(Years)

def final_rec_sheet(Record_Status, Record_Sums,
    final_cols = ['Total Days',
                     'City Business Days',
                     'Rounds',
                     'Days till Routing',
                     'Days per Round',
                     'Submittal Date',
                     'Final Date']):
    
    # Add Planning Fields
    if 'Staff Analysis - Set for Hearing' in Record_Status.index.get_level_values(1):
        Planning = True
        
        Comm_cols = ['Commissions','Comm List', 'Comm Days']
        
        # Add Commission Rounds
        Record_Sums = Commission_Counts(Record_Status, Record_Sums)
        final_cols.extend(Comm_cols)
        
        # Add Commission Rounds
        Record_Sums = Add_Commission_to_Total(Record_Status, Record_Sums)
    
    # Only Selecting 2018-2020 Closed Applications
    """
    Record_Sums = Record_Sums[(Record_Sums['Final Date'].dt.to_period('Q') > less)&(Record_Sums['Final Date'].dt.to_period('Q') < great)]
    """
    # Cleaned up lengths
    Record_Sums['Length of Process'] = Record_Sums['Total Days'].apply(lambda td: Process_Length(td))
    final_cols.append('Length of Process')
    
    # Final Format
    Record_Sums = Record_Sums[final_cols].join(Admin_Permits, how='inner')
    
    ### ADD APN PROCESS
    
    return Record_Sums, final_cols

#if ('Planning' in tit)and('DONT' in tit):
    #Records, final_cols = final_rec_sheet(Formatted_Permits, Records, final_cols=keep_fields)
admin_cols = [
        c for c in list(Admin_Permits) #if c not in ['Permit Type', 'Record Status']
    ]
if len(admin_cols) > 0:
    Records = Records.set_index(index).join(Admin_Permits[admin_cols])
if 'CONST TYPE CODE' in admin_cols:
    path = r'O:\CDD\PLANNING\AN\Projects\Accela Reporting\Building\ConstCodes.csv'
    const_dict = pd.read_csv(path)[['Code','Descript']].set_index('Code')['Descript'].to_dict()
    #Records['Construct'] = Records['CONST TYPE CODE'].map(const_dict)
    
    #resi = [101, 102, 103, 104, 105, 106, 109, 434]
    #Records = Records[Records['CONST TYPE CODE'].isin(resi)]

date_cols = [col for col in list(Records) if 'date' in col.lower()]
Records[date_cols] = Records[date_cols].apply(lambda x: x.dt.strftime('%Y-%m-%d')).replace(np.nan, '')

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
bad_cols_to_drop = ['E_ID', 'Days per Round']
rec = Records[[l for l in list(Records) if l not in bad_cols_to_drop]] \
    .copy() \
    .drop_duplicates() \
    .copy() \
    .reset_index() \
    .rename(columns={'index':'Application #'})

rec.head()

,Application #,Total Days,Total Business Days,City Business Days,Rounds,Days till Routing,Submittal Date,Final Date,Record Type,Permit Type,...,Num of Entitlements,Staff Decision,Board Decision,Boards,Board Count,Task Status,Board Total Days,Board Business Days,Board Start Date,Board End Date
0,Y18-001,78,54,36,2,1,2018-01-03,2018-03-21,Planning Project,Design Review Residential,...,1,True,False,,0,Intake Review - Application Accepted,0,0,,
1,Y18-004,91,64,64,1,0,2018-01-23,2018-04-23,Planning Project,Design Review Commercial,...,2,True,False,,0,Intake Review - Application Accepted,0,0,,
2,Y18-005,169,119,31,1,0,2018-01-24,2018-07-11,Planning Project,Use Permit Conditional,...,1,False,True,PC,1,Intake Review - Application Accepted,30,22,2018-03-28,2018-04-26
3,Y18-009,722,487,20,2,1,2018-02-16,2020-02-07,Planning Project,Design Review Other,...,1,True,False,,0,Intake Review - Application Accepted,0,0,,
4,Y18-012,175,124,109,2,2,2018-02-27,2018-08-20,Planning Project,Design Review Commercial,...,1,False,True,"DRC, CC, ZA",3,Intake Review - Application Accepted,1,1,2018-08-20,2019-05-08


## 4.3 Export

In [ ]:
save_location = r'data\clean_data\planning_permit_metrics.csv'
rec.to_csv(save_location)